# 5章 誤差逆伝播法

- 重みパラメータに関する損失関数の勾配は数値微分によって求めた
- 実装は簡単だけど、時間がかかる難点がある
- 重みパラメータの勾配計算を効率良く行う「誤差逆伝播法」を学ぶ
- 誤差逆伝播法の理解には数式と計算グラフが良さそう（筆者的に）
- この章では計算グラフで説明する
- 計算グラフによって説明するアイデアは↓を参考にしてるらしい
- http://karpathy.github.io/neuralnets/
- http://cs231n.stanford.edu/

In [7]:
from sympy import *

z = Function("z")
t = Symbol("t")
x = Symbol("x")
y = Symbol("y")
z = (x + y) ** 2
t = (x + y)


In [8]:
diff(z, x)

2*x + 2*y

In [9]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy


In [10]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)
print(price)

# backward

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)


220.00000000000003
2.2 110.00000000000001 200


次回

## 5.4.2 加算レイヤの実装

デュエルスタンバイ